In [ ]:
# Let's examine the actual BRCA Exchange In Silico Prior Probability columns
print("=== EXAMINING BRCA EXCHANGE IN SILICO PRIOR PROBABILITIES ===")

# Look for the actual pathogenicity prior columns
prior_cols = [col for col in df.columns if 'pathogenicity' in col.lower() and 'prior' in col.lower()]
print("Available pathogenicity prior columns:")
for col in prior_cols:
    print(f"  - {col}")

# Examine the main pathogenicity prior column
print(f"\n=== PATHOGENICITY_PRIOR COLUMN ===")
pathogenicity_prior = df['pathogenicity_prior']
print(f"Total values: {len(pathogenicity_prior)}")
print(f"Non-null values: {pathogenicity_prior.notna().sum()}")
print(f"Unique values (first 20): {pathogenicity_prior.value_counts().head(20)}")

# Check protein and splicing priors separately
print(f"\n=== PROTEIN PRIOR ===")
protein_prior = df['pathogenicity_protein_prior']
protein_numeric = pd.to_numeric(protein_prior, errors='coerce')
print(f"Non-null numeric values: {protein_numeric.notna().sum()}")
if protein_numeric.notna().sum() > 0:
    print(f"Range: {protein_numeric.min():.3f} to {protein_numeric.max():.3f}")
    print(f"Mean: {protein_numeric.mean():.3f}")

print(f"\n=== SPLICING PRIORS ===")
splicing_cols = ['pathogenicity_splicing_prior_de_novo', 
                'pathogenicity_splicing_prior_ref_acceptor', 
                'pathogenicity_splicing_prior_ref_donor']

for col in splicing_cols:
    if col in df.columns:
        splicing_prior = df[col]
        splicing_numeric = pd.to_numeric(splicing_prior, errors='coerce')
        print(f"{col}:")
        print(f"  Non-null numeric values: {splicing_numeric.notna().sum()}")
        if splicing_numeric.notna().sum() > 0:
            print(f"  Range: {splicing_numeric.min():.3f} to {splicing_numeric.max():.3f}")
            print(f"  Mean: {splicing_numeric.mean():.3f}")

In [ ]:
# Re-load the data and examine the proper BRCA Exchange pathogenicity priors
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Create outputs directory if it doesn't exist
Path("./outputs").mkdir(exist_ok=True)

# Read the variants data
df = pd.read_csv("~/brca_test_data/output/variants_output.tsv", sep='\t', low_memory=False)

print("=== EXAMINING BRCA EXCHANGE IN SILICO PRIOR PROBABILITIES ===")

# Look for the actual pathogenicity prior columns
prior_cols = [col for col in df.columns if 'pathogenicity' in col.lower() and 'prior' in col.lower()]
print("Available pathogenicity prior columns:")
for col in prior_cols:
    print(f"  - {col}")

# Examine the main pathogenicity prior column
print(f"\n=== PATHOGENICITY_PRIOR COLUMN ===")
pathogenicity_prior = df['pathogenicity_prior']
print(f"Total values: {len(pathogenicity_prior)}")
print(f"Non-null values: {pathogenicity_prior.notna().sum()}")
print(f"Unique values (first 20): {pathogenicity_prior.value_counts().head(20)}")

# Check protein and splicing priors separately
print(f"\n=== PROTEIN PRIOR ===")
protein_prior = df['pathogenicity_protein_prior']
protein_numeric = pd.to_numeric(protein_prior, errors='coerce')
print(f"Non-null numeric values: {protein_numeric.notna().sum()}")
if protein_numeric.notna().sum() > 0:
    print(f"Range: {protein_numeric.min():.3f} to {protein_numeric.max():.3f}")
    print(f"Mean: {protein_numeric.mean():.3f}")

print(f"\n=== SPLICING PRIORS ===")
splicing_cols = ['pathogenicity_splicing_prior_de_novo', 
                'pathogenicity_splicing_prior_ref_acceptor', 
                'pathogenicity_splicing_prior_ref_donor']

for col in splicing_cols:
    if col in df.columns:
        splicing_prior = df[col]
        splicing_numeric = pd.to_numeric(splicing_prior, errors='coerce')
        print(f"{col}:")
        print(f"  Non-null numeric values: {splicing_numeric.notna().sum()}")
        if splicing_numeric.notna().sum() > 0:
            print(f"  Range: {splicing_numeric.min():.3f} to {splicing_numeric.max():.3f}")
            print(f"  Mean: {splicing_numeric.mean():.3f}")

# Let's also check what the actual pathogenicity_prior values look like
print(f"\n=== SAMPLE PATHOGENICITY_PRIOR VALUES ===")
# Convert to numeric and see the distribution
pathogenicity_numeric = pd.to_numeric(pathogenicity_prior, errors='coerce')
print(f"Numeric pathogenicity_prior values: {pathogenicity_numeric.notna().sum()}")
if pathogenicity_numeric.notna().sum() > 0:
    print(f"Range: {pathogenicity_numeric.min():.3f} to {pathogenicity_numeric.max():.3f}")
    print(f"Mean: {pathogenicity_numeric.mean():.3f}")
    print(f"Distribution:")
    print(pathogenicity_numeric.describe())

In [ ]:
# Now create the CORRECTED analysis using proper BRCA Exchange In Silico Prior Probabilities

def clinical_to_numeric(clinical_sig):
    """Convert clinical significance to numeric scale (0=Benign, 4=Pathogenic)"""
    if pd.isna(clinical_sig) or clinical_sig == '-':
        return np.nan
    
    clinical_sig = str(clinical_sig).lower()
    
    if 'benign' in clinical_sig and 'likely' not in clinical_sig:
        return 0  # Benign
    elif 'likely benign' in clinical_sig:
        return 1  # Likely Benign
    elif 'uncertain' in clinical_sig or 'vus' in clinical_sig:
        return 2  # Uncertain/VUS
    elif 'likely pathogenic' in clinical_sig:
        return 3  # Likely Pathogenic
    elif 'pathogenic' in clinical_sig and 'likely' not in clinical_sig:
        return 4  # Pathogenic
    else:
        return np.nan

# Create corrected analysis dataframe
corrected_df = df[['clinical_significance_enigma', 'pathogenicity_prior', 
                   'pathogenicity_protein_prior', 'pathogenicity_splicing_prior_de_novo',
                   'pathogenicity_splicing_prior_ref_acceptor', 'pathogenicity_splicing_prior_ref_donor',
                   'gene_symbol', 'genomic_hgvs_38', 'variant_type']].copy()

# Convert clinical significance to numeric
corrected_df['clinical_numeric'] = corrected_df['clinical_significance_enigma'].apply(clinical_to_numeric)

# Use the ACTUAL BRCA Exchange In Silico Prior Probability of Pathogenicity
corrected_df['insilico_prior'] = pd.to_numeric(corrected_df['pathogenicity_prior'], errors='coerce')

# Remove rows with missing data for both clinical and in-silico
corrected_clean = corrected_df.dropna(subset=['clinical_numeric', 'insilico_prior'])

print("=== CORRECTED ANALYSIS USING PROPER BRCA EXCHANGE METHODOLOGY ===")
print(f"Total variants: {len(df)}")
print(f"Variants with both clinical significance and BRCA Exchange In Silico Prior: {len(corrected_clean)}")

print(f"\n=== BRCA EXCHANGE IN SILICO PRIOR DISTRIBUTION ===")
print(f"Range: {corrected_clean['insilico_prior'].min():.3f} to {corrected_clean['insilico_prior'].max():.3f}")
print(f"Mean: {corrected_clean['insilico_prior'].mean():.3f}")
print(f"Median: {corrected_clean['insilico_prior'].median():.3f}")

# Show distribution by ranges
def categorize_insilico_prior(prior):
    """Categorize in-silico prior probability"""
    if prior < 0.1:
        return 'Very Low (< 0.1)'
    elif prior < 0.3:
        return 'Low (0.1-0.3)'
    elif prior < 0.5:
        return 'Moderate (0.3-0.5)'
    elif prior < 0.7:
        return 'High (0.5-0.7)'
    else:
        return 'Very High (≥ 0.7)'

corrected_clean['insilico_category'] = corrected_clean['insilico_prior'].apply(categorize_insilico_prior)

print(f"\nIn-silico prior probability categories:")
insilico_dist = corrected_clean['insilico_category'].value_counts()
for category, count in insilico_dist.items():
    percentage = (count / len(corrected_clean)) * 100
    print(f"  {category}: {count} variants ({percentage:.1f}%)")

print(f"\nClinical significance distribution:")
clinical_dist = corrected_clean['clinical_numeric'].value_counts().sort_index()
clinical_labels = ['Benign', 'Likely Benign', 'Uncertain/VUS', 'Likely Pathogenic', 'Pathogenic']
for idx, count in clinical_dist.items():
    percentage = (count / len(corrected_clean)) * 100
    print(f"  {clinical_labels[int(idx)]}: {count} variants ({percentage:.1f}%)")

In [ ]:
# Create the CORRECTED bivariate scatter plot using proper BRCA Exchange In Silico Prior Probabilities

plt.style.use('default')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))

# Add jitter for better visualization
np.random.seed(42)
jitter_strength = 0.05
corrected_clean['clinical_jitter'] = corrected_clean['clinical_numeric'] + np.random.normal(0, jitter_strength, len(corrected_clean))
corrected_clean['insilico_jitter'] = corrected_clean['insilico_prior'] + np.random.normal(0, jitter_strength/10, len(corrected_clean))

# Plot 1: Scatter plot - Clinical vs BRCA Exchange In Silico Prior Probability
scatter = ax1.scatter(corrected_clean['clinical_jitter'], corrected_clean['insilico_jitter'], 
                     alpha=0.6, s=25, c=corrected_clean['insilico_prior'], 
                     cmap='RdYlBu_r', vmin=0, vmax=1)

ax1.set_xlabel('Clinical Pathogenicity Assessment', fontsize=12, fontweight='bold')
ax1.set_ylabel('BRCA Exchange In Silico Prior Probability', fontsize=12, fontweight='bold')
ax1.set_title('Clinical vs BRCA Exchange In Silico Prior Probability\n(Corrected Analysis)', 
              fontsize=14, fontweight='bold')

ax1.set_xlim(-0.5, 4.5)
ax1.set_ylim(-0.05, 1.05)
ax1.set_xticks(range(5))
ax1.set_xticklabels(clinical_labels, rotation=45, ha='right')
ax1.grid(True, alpha=0.3)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('In Silico Prior Probability', fontsize=10)

# Plot 2: Box plot showing distribution by clinical category
clinical_categories = []
insilico_values = []
for i in range(5):
    mask = corrected_clean['clinical_numeric'] == i
    if mask.sum() > 0:
        clinical_categories.extend([clinical_labels[i]] * mask.sum())
        insilico_values.extend(corrected_clean[mask]['insilico_prior'].tolist())

box_df = pd.DataFrame({'Clinical': clinical_categories, 'InSilico_Prior': insilico_values})
sns.boxplot(data=box_df, x='Clinical', y='InSilico_Prior', ax=ax2)
ax2.set_xlabel('Clinical Pathogenicity Assessment', fontsize=12, fontweight='bold')
ax2.set_ylabel('BRCA Exchange In Silico Prior Probability', fontsize=12, fontweight='bold')
ax2.set_title('Distribution of In Silico Priors by Clinical Category', fontsize=14, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Plot 3: Correlation analysis with trend line
from scipy import stats
correlation, p_value = stats.pearsonr(corrected_clean['clinical_numeric'], corrected_clean['insilico_prior'])

ax3.scatter(corrected_clean['clinical_numeric'], corrected_clean['insilico_prior'], 
           alpha=0.5, s=20, color='steelblue')

# Add trend line
z = np.polyfit(corrected_clean['clinical_numeric'], corrected_clean['insilico_prior'], 1)
p = np.poly1d(z)
ax3.plot(range(5), p(range(5)), "r--", alpha=0.8, linewidth=2)

ax3.set_xlabel('Clinical Pathogenicity Assessment', fontsize=12, fontweight='bold')
ax3.set_ylabel('BRCA Exchange In Silico Prior Probability', fontsize=12, fontweight='bold')
ax3.set_title(f'Correlation Analysis\nr = {correlation:.3f}, p = {p_value:.2e}', 
              fontsize=14, fontweight='bold')
ax3.set_xlim(-0.5, 4.5)
ax3.set_ylim(-0.05, 1.05)
ax3.set_xticks(range(5))
ax3.set_xticklabels(clinical_labels, rotation=45, ha='right')
ax3.grid(True, alpha=0.3)

# Plot 4: Heatmap of agreement categories
def get_agreement_category_corrected(clinical, insilico):
    """Categorize agreement between clinical and in-silico predictions"""
    # Convert clinical to probability-like scale
    clinical_prob_map = {0: 0.01, 1: 0.25, 2: 0.5, 3: 0.75, 4: 0.99}
    clinical_prob = clinical_prob_map[clinical]
    
    diff = abs(clinical_prob - insilico)
    
    if diff <= 0.2:
        return 'Strong Agreement'
    elif diff <= 0.4:
        return 'Moderate Agreement'
    elif diff <= 0.6:
        return 'Weak Agreement'
    else:
        return 'Strong Disagreement'

corrected_clean['agreement_category'] = corrected_clean.apply(
    lambda row: get_agreement_category_corrected(row['clinical_numeric'], row['insilico_prior']), axis=1
)

# Create heatmap data
heatmap_data = corrected_clean.pivot_table(
    values='gene_symbol', 
    index='insilico_category', 
    columns='clinical_numeric', 
    aggfunc='count', 
    fill_value=0
)

sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='Blues', ax=ax4, 
            cbar_kws={'label': 'Number of Variants'})

ax4.set_xlabel('Clinical Pathogenicity Assessment', fontsize=12, fontweight='bold')
ax4.set_ylabel('In Silico Prior Probability Category', fontsize=12, fontweight='bold')
ax4.set_title('Variant Count Heatmap\n(Clinical vs In Silico Categories)', 
              fontsize=14, fontweight='bold')
ax4.set_xticklabels(clinical_labels, rotation=45, ha='right')

plt.tight_layout()
plt.savefig('./outputs/corrected_clinical_vs_brca_exchange_insilico.png', 
            dpi=300, bbox_inches='tight')
plt.show()

# Print corrected summary statistics
print("=== CORRECTED SUMMARY STATISTICS ===")
print(f"Total variants analyzed: {len(corrected_clean)}")
print(f"Pearson correlation coefficient: {correlation:.3f} (p-value: {p_value:.2e})")

print(f"\n=== AGREEMENT ANALYSIS ===")
agreement_counts = corrected_clean['agreement_category'].value_counts()
for category, count in agreement_counts.items():
    percentage = (count / len(corrected_clean)) * 100
    print(f"  {category}: {count} variants ({percentage:.1f}%)")

# Show examples of strong disagreements
print(f"\n=== EXAMPLES OF STRONG DISAGREEMENTS ===")
strong_disagreements = corrected_clean[corrected_clean['agreement_category'] == 'Strong Disagreement']
if len(strong_disagreements) > 0:
    print(f"Found {len(strong_disagreements)} variants with strong disagreement:")
    for idx, row in strong_disagreements.head(5).iterrows():
        clinical_cat = clinical_labels[int(row['clinical_numeric'])]
        print(f"  {row['genomic_hgvs_38']}: Clinical={clinical_cat}, In-silico Prior={row['insilico_prior']:.3f}")
else:
    print("No variants with strong disagreement found.")

In [ ]:
# Create a comprehensive methodology comparison document

print("=" * 80)
print("METHODOLOGY EXPLANATION: CLINICAL vs IN-SILICO PATHOGENICITY PREDICTIONS")
print("=" * 80)

print("\n## WHAT I INITIALLY DID (INCORRECT APPROACH):")
print("=" * 50)
print("1. **Composite Score Creation**: I created my own 'in-silico pathogenicity score' by combining:")
print("   - SpliceAI scores (0-1 scale) converted to 0-4 categorical scale")
print("   - Functional assay results (FUNC/INT/LOF) mapped to categories")
print("   - Used a 'highest impact' rule to combine them")
print("\n2. **Problems with this approach**:")
print("   - Not using the official BRCA Exchange methodology")
print("   - Oversimplified conversion of continuous scores to categories")
print("   - Missing the sophisticated protein/splicing prior calculations")
print("   - Not accounting for variant type and location properly")

print("\n## WHAT I SHOULD HAVE DONE (CORRECTED APPROACH):")
print("=" * 50)
print("1. **Used BRCA Exchange In Silico Prior Probability of Pathogenicity**:")
print("   - This is the official, pre-calculated score in the 'pathogenicity_prior' column")
print("   - Range: 0.02 to 0.99 (probability scale)")
print("   - Already incorporates the max(protein_prior, splicing_prior) methodology")
print("\n2. **BRCA Exchange Methodology** (as you described):")
print("   - **Protein-level estimation**: Based on variant type, location, and impact on protein function")
print("   - **Splicing-level estimation**: Considers impact on mRNA splicing (de novo, acceptor, donor sites)")
print("   - **Final score**: Takes the MAXIMUM of protein and splicing estimations")
print("   - **Calibrated**: Using clinical datasets of known pathogenic/benign variants")

print("\n## KEY DIFFERENCES IN RESULTS:")
print("=" * 50)
print("### My Initial (Incorrect) Analysis:")
print(f"   - Variants analyzed: 7,310")
print(f"   - Correlation: 0.135 (very weak)")
print(f"   - Strong disagreement: 62.0%")
print(f"   - Perfect agreement: 21.0%")

print("\n### Corrected Analysis (Using BRCA Exchange Priors):")
print(f"   - Variants analyzed: 2,925")
print(f"   - Correlation: 0.961 (very strong!)")
print(f"   - Strong disagreement: 0.5%")
print(f"   - Strong agreement: 58.5%")

print("\n## WHY THE HUGE DIFFERENCE?")
print("=" * 50)
print("1. **Proper Calibration**: BRCA Exchange priors are calibrated against clinical data")
print("2. **Sophisticated Modeling**: Accounts for variant location, type, and multiple mechanisms")
print("3. **Expert Curation**: Incorporates domain expertise in BRCA variant interpretation")
print("4. **Smaller Dataset**: Only 2,925 variants have both clinical significance AND in-silico priors")
print("   (vs 7,310 in my crude analysis)")

print("\n## CLINICAL IMPLICATIONS:")
print("=" * 50)
print("✅ **BRCA Exchange In Silico Priors are highly predictive** of clinical significance")
print("✅ **Strong correlation (r=0.961)** suggests the methodology is working well")
print("✅ **Low disagreement rate (0.5%)** indicates good alignment with clinical assessment")
print("⚠️  **However**: These are still 'priors' and require additional clinical/functional evidence")

print("\n## EXAMPLES OF THE FEW DISAGREEMENTS:")
print("=" * 50)
disagreement_examples = [
    "NC_000013.11:g.32319330G>A: Clinical=Pathogenic, In-silico Prior=0.340",
    "NC_000013.11:g.32319330G>C: Clinical=Pathogenic, In-silico Prior=0.340", 
    "NC_000013.11:g.32346896G>A: Clinical=Pathogenic, In-silico Prior=0.340",
    "NC_000013.11:g.32362596A>T: Clinical=Pathogenic, In-silico Prior=0.290",
    "NC_000013.11:g.32363196A>G: Clinical=Benign, In-silico Prior=0.810"
]

for example in disagreement_examples:
    print(f"   - {example}")

print("\n## CONCLUSION:")
print("=" * 50)
print("The BRCA Exchange In Silico Prior Probability of Pathogenicity is a sophisticated,")
print("well-calibrated tool that shows excellent agreement with clinical assessments.")
print("My initial approach was overly simplistic and didn't capture the nuanced")
print("methodology that BRCA Exchange uses, which explains the dramatic difference in results.")

print("\n" + "=" * 80)
print("LESSON: Always use the official, validated scores when available!")
print("=" * 80)